# Neurostatistics 2025: Circular (directional) statistics practical

_Kira M. Dusterwald, February 2025_

In this tutorial, you will learn:
* The difference between intrinsic and extrinsic circular means, and how these calculations compare on data (for notes on this, see the lecture notes).
* How to use maximum likelihood estimation to fit the parameters of a von Mises distribution to the neural activity of a tuning curve from V1.

In [1]:
import numpy as np
import scipy.stats as ss
import pickle as pkl

## Dataset

Load this dictionary containing two neural activity arrays, each of which is from a neuron in primary visual cortex (V1) selected from a mouse from the dataset by _Stringer et al., Nature, 2019_. The neurons have keys "neur_1" and "neur_2".

The neural activity is derived from calcium imaging flourescence to 32 different orientations, evenly spaced around the circle, with the last channel corresponding to spontaneous activity. The orientations are shown to the animal over 64 repeats, and I have calculated the average activity for each orientation across repeats.

In [8]:
neurons = pkl.load(open('vm_neurons.obj','rb'))
print(neurons.keys())

dict_keys(['neur_1', 'neur_2'])


### Task 1: Plot rose or polar plots of the data. 

Represent the data either proportional to activity, or as a collection of points on the unit circle only.

## Circular means and fundamental statistical tests

### Task 2: Calculate the circular mean in different ways.

1. Code up the linear mean as we are used to.
2. Code up the extrinsic definition of the mean. Also code up the mean resultant vector length.
3. The intrinsic definition (minimisation with complex number form for angles) is built-in to scipy. You can use that function directly.

Now compare these different types of mean on the two neurons. For which neuron do these definitions agree most?

What does the mean resultant vector look like for each set of activities?

### Task 3: Use the Rayleigh test for uniformity.

## Maximum likelihood estimation procedure to fit tuning curves using von Mises distribution

### Task 4: Use the MLE derivation to fit the parameters for von Mises tuning curves to the two neurons. 

When is the fit good or not? 

Bonus! How does the analytical solution compare to the solution from a numerical solver?